In [1]:
import os
import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim 

from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box
import resnet_no_fc

from collections import OrderedDict

# Threat score for road detection
from helper import compute_ats_bounding_boxes, compute_ts_road_map

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

In [3]:
# Image folder
image_folder = '/scratch/brs426/data'
annotation_csv = '/scratch/brs426/data/annotation.csv'

# Indices
train_labeled_scene_index = np.arange(106, 129)
val_labeled_scene_index = np.arange(129, 132)
test_labeled_scene_index = np.arange(132, 134)

### Load in Self-Supervised Models

In [4]:
path_to_ssl_model = "/scratch/brs426/ben_models/resnet50_no_bn_jigsaw_single_image_hard_200_permutations_50_bonus_SSL_checkpoint.p"

### Model Architectures - Bins

In [5]:
class RoadSegmentation(nn.Module):
    def __init__(self, num_classes):
        super(RoadSegmentation, self).__init__()
        self.resnet = torchvision.models.resnet50()
        self.fc1 = nn.Linear(6 * 1000, num_classes)
        
    def forward(self, x):
        batch_size = x.shape[0]
        num_images = x.shape[1]
        channels = x.shape[2]
        height = x.shape[3]
        width = x.shape[4]
        
        # Reshape to feed in images
        x = x.reshape(-1, channels, height, width)
        
        # Feed into ResNet
        x = self.resnet(x)
        
        # Go to 256 per image
        
        # Reshape to get batch back
        x = x.reshape(batch_size, -1)
        
        # Feed through fc and sigmoid to get probabilities
        x = torch.sigmoid(self.fc1(x))
        
        return x
    
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        
        self.encoder = torchvision.models.resnet50()
        self.encoder.fc = nn.Identity()
        self.concat_dim = 256 * 6
        
        self.compress = nn.Sequential(OrderedDict([
            ('linear0', nn.Linear(2048, 256)),
            ('drop', nn.Dropout(p = 0.5)),
            ('relu', nn.ReLU()),
        ]))
        
        self.segmentation = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(self.concat_dim, 25600)),
            ('sigmoid', nn.Sigmoid())
        ]))
        
    def forward(self, x):
        batch_size = x.shape[0]
        num_images = x.shape[1]
        channels = x.shape[2]
        height = x.shape[3]
        width = x.shape[4]
        
        # Reshape to feed in images
        x = x.reshape(-1, channels, height, width)
        
        x = self.encoder(x)
        x = self.compress(x)
        x = x.view(-1, self.concat_dim)
        return self.segmentation(x)

### Load in model and match weights

In [7]:
ssl_checkpoint = torch.load(path_to_ssl_model)
model = SimpleModel().to(device)

In [8]:
ssl_weights = ssl_checkpoint['model_state_dict']

In [9]:
# Load in resnet weights 
resnet_dict = model.encoder.state_dict()
for weight in ssl_weights:
    if weight[7:] in model.encoder.state_dict():
        resnet_dict[weight[7:]].data.copy_(ssl_weights[weight])

### Data Loading

In [10]:
def segmentation_collate_fn(batch):
    BLOCK_SIZE = 5
    road_maps = []
    road_bins = []
    images = []
    for x in batch:
        
        grid = []
        # Collect six images for this sample. 
        six_images = []
        for i in range(6):
            six_images.append(torch.as_tensor(x[0][i]))
        
        # Get road_image and cast it to float
        road_image = torch.as_tensor(x[2])
        road_maps.append(road_image)
        road_image = road_image.float()
        
        for x in range(0, 800, BLOCK_SIZE):
            for y in range(0, 800, BLOCK_SIZE):
                block = road_image[x:x+BLOCK_SIZE, y:y+BLOCK_SIZE]
                score = torch.sum(block).item()
                # If more than have the pixels are 1, classify as road
                if score > (BLOCK_SIZE**2) / 2:
                    grid.append(1.0)
                else:
                    grid.append(0.0)
            
        images.append(torch.stack(six_images))
                
        road_bins.append(torch.as_tensor(grid))
                
    boom = torch.stack(images), torch.stack(road_bins), torch.stack(road_maps)
    return boom

In [11]:
transform = torchvision.transforms.ToTensor()
aug_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomApply([
        torchvision.transforms.ColorJitter(brightness = 0.5, contrast = 0.5, saturation = 0.4, hue = (-0.5, 0.5)),
        torchvision.transforms.Grayscale(3),
#         transforms.RandomAffine(3),
    ]),
    torchvision.transforms.ToTensor(),
])

In [12]:
transform = torchvision.transforms.ToTensor()

labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=train_labeled_scene_index,
                                  transform=aug_transform,
                                  extra_info=True
                                 )
trainloader = torch.utils.data.DataLoader(labeled_trainset, batch_size=4, shuffle=True, num_workers=10, collate_fn=segmentation_collate_fn)

labeled_valset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=val_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
valloader = torch.utils.data.DataLoader(labeled_valset, batch_size=1, shuffle=True, num_workers=2, collate_fn=segmentation_collate_fn)

labeled_testset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=test_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
testloader = torch.utils.data.DataLoader(labeled_testset, batch_size=2, shuffle=True, num_workers=2, collate_fn=segmentation_collate_fn)

### Training Logic

In [13]:
# Train logic, return average loss over training set after each epoch
def train(model, device, train_loader, optimizer, epoch, log_file_path, log_interval = 250):
    # Set model to training mode
    model.train()
    
    # Number correct for accuracy
    num_correct = 0
    
    # Train loss
    train_loss = 0
    
    f = open(log_file_path, "a+")
    # Loop through examples
    for batch_idx, (images, bins, road_map) in enumerate(train_loader):
        
        
        # Send data and target to device
        data, target = images.to(device), bins.to(device)
        
        # Zero out optimizer
        optimizer.zero_grad()
        
        # Pass data through model - right now only segmentation
        output = model(data)
        # Should be batch_size X 800 X 800
        output = output.squeeze()
        
        # Compute the loss
        loss = F.binary_cross_entropy(output, target)
        train_loss += loss.item()
        
        # Backpropagate loss
        loss.backward()
        
        # Make a step with the optimizer
        optimizer.step()
        
        # Print loss (uncomment lines below once implemented)
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            f.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\n'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    
    # Average train loss
    average_train_loss = train_loss / len(train_loader)
    # Print loss (uncomment lines below once implemented)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        average_train_loss, num_correct, len(train_loader.dataset),
        100. * num_correct / len(train_loader.dataset)))
    f.write('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        average_train_loss, num_correct, len(train_loader.dataset),
        100. * num_correct / len(train_loader.dataset)))
    f.close()

### Validation Logic

In [14]:
def reconstruct_from_bins(bins, block_size, threshold):
    road_map = torch.zeros((800, 800))
    idx = 0
    for x in range(0, 800, block_size):
        for y in range(0, 800, block_size):
            road_map[x:x+block_size, y:y+block_size] = bins[idx]
            idx += 1
    return road_map > threshold


# Define test method
def test(model, device, test_loader, log_file_path):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    thresholds = [0.42]
    
    threat_scores = torch.zeros(1)
    
    f = open(log_file_path, "a+")
    # don't need autograd for eval
    with torch.no_grad():
        for batch_idx, (images, bins, road_map) in enumerate(test_loader):

            # Send data and target to device
            data, target= images.to(device), bins.to(device)

            # Pass data through model - right now only segmentation
            output = model(data)
            # Should be batch_size X 6400

            # Compute the loss
            loss = F.binary_cross_entropy(output, target)
            test_loss += loss.item()
            
            # Now squeeze for reconstruction
            output = output.squeeze()
            
            # Compute threat score at 4 different thresholds
            for idx in range(len(thresholds)):
                reconstructed_road_map = reconstruct_from_bins(output, 5, thresholds[idx]).cpu()
                ts_road_map = compute_ts_road_map(reconstructed_road_map, road_map)
                threat_scores[idx] += ts_road_map
         
    # Compute the average test_loss
    # avg_test_loss = TODO
    avg_test_loss = test_loss / len(test_loader)
    
    # Compute average threat scores
    avg_threat_scores = threat_scores / len(test_loader)
    
    print('\Threat scores: \t {}:{}\n'.format(thresholds[0], avg_threat_scores[0]))
    # Print loss (uncomment lines below once implemented)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))
    
    f.write('\Threat scores: \t {}:{}\n'.format(thresholds[0], avg_threat_scores[0]))
    f.write('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))
    f.close()
    
    return avg_test_loss, avg_threat_scores[0]

In [15]:
#Freeze everything but batch norm and last resnet layer
for name, param in model.encoder.named_parameters():
    if name.find("bn") >= 0 or name.find("layer4") >= 0:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [16]:
model = model.to(device)

# Optimizer
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4)
# Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.025, patience=1)

In [ ]:
f = "/scratch/brs426/ben_models/simple_model_pretrained_permutations_block_size_5_test_threat.log"
best_val_threat = None
save_path = "/scratch/brs426/ben_models/simple_model_pretrained_permutations_block_size_5_test_threat.p"
epochs = 20
for epoch in range(21, 41):
    # Train model
    loss = train(model, device, trainloader, optimizer, epoch, f)
    val_loss, val_threat = test(model, device, valloader, f)
    scheduler.step(val_threat)
    #Save model
    if best_val_threat is None:
        torch.save({'model_state_dict': model.state_dict(),
                   'optimizer_state_dict': optimizer.state_dict(),
                   'threat': val_threat}, save_path)
        best_val_threat = val_threat
    elif val_threat > best_val_threat:
        torch.save({'model_state_dict': model.state_dict(),
                   'optimizer_state_dict': optimizer.state_dict(),
                   'threat': val_threat}, save_path)
        best_val_threat = val_threat

Train Epoch: 21 [0/2898 (0%)]	Loss: 0.706351
Train Epoch: 21 [1000/2898 (34%)]	Loss: 0.442537
Train Epoch: 21 [2000/2898 (69%)]	Loss: 0.309112

Train set: Average loss: 0.3844, Accuracy: 0/2898 (0%)

\Threat scores: 	 0.42:0.6969773173332214


Test set: Average loss: 0.4122, Accuracy: 0/378 (0%)

Train Epoch: 22 [0/2898 (0%)]	Loss: 0.448144
Train Epoch: 22 [1000/2898 (34%)]	Loss: 0.472196
Train Epoch: 22 [2000/2898 (69%)]	Loss: 0.182180

Train set: Average loss: 0.3178, Accuracy: 0/2898 (0%)

\Threat scores: 	 0.42:0.704693078994751


Test set: Average loss: 0.4126, Accuracy: 0/378 (0%)

Train Epoch: 23 [0/2898 (0%)]	Loss: 0.359155
Train Epoch: 23 [1000/2898 (34%)]	Loss: 0.241734
Train Epoch: 23 [2000/2898 (69%)]	Loss: 0.166951

Train set: Average loss: 0.2882, Accuracy: 0/2898 (0%)



### Threshold Experiments

In [24]:
# Predicting everything as road
model.eval()

thresholds = [0.3]

threat_scores = torch.zeros(1)

with torch.no_grad():
    for batch_idx, (images, bins, road_map) in enumerate(valloader):

        # Send data and target to device
        data, target= images.to(device), bins.to(device)

        # Pass data through model - right now only segmentation
        output = model(data)
        # Should be batch_size X 6400

#         # Compute the loss
#         loss = F.binary_cross_entropy(output, target)
#         test_loss += loss.item()

        # Now squeeze for reconstruction
        output = output.squeeze()

        # Compute threat score at 4 different thresholds
        for idx in range(len(thresholds)):
            reconstructed_road_map = reconstruct_from_bins(output, 10, thresholds[idx]).cpu()
            ts_road_map = compute_ts_road_map(reconstructed_road_map, road_map)
            threat_scores[idx] += ts_road_map

avg_threat_scores = threat_scores / len(valloader)
print("Threshold 0.5: {}".format(avg_threat_scores))

Threshold 0.5: tensor([0.6966])
